In [1]:
import tensorflow as tf
from tensorflow.keras import layers

import numpy as np
import gensim 
import gensim.models as g
from gensim.models import Word2Vec
from gensim.utils import tokenize
import sklearn
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import word2vec

2023-03-19 23:57:49.066705: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-19 23:57:49.239570: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-19 23:57:49.239598: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-19 23:57:50.398693: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
data = (pd.read_csv("data/daniel.csv")
        .loc[:,"Contents"]
        .dropna(how="any")
        .apply(lambda x: x.lower())
        #.apply(tokenize)
        #.apply(list)
        .tolist()
       )
data = tf.data.Dataset.from_tensor_slices(data)

2023-03-19 23:58:00.700215: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-19 23:58:00.700275: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (danilka): /proc/driver/nvidia/version does not exist
2023-03-19 23:58:00.701299: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
pip uninstall tensorflow

In [ ]:
pip install --upgrade tensorflow